In [1]:
all = [var for var in globals() if var[0] != "_"]
for var in all:
    del globals()[var]

In [2]:
import numpy as np
import pandas as pd
import random
import calendar
import matplotlib.pyplot as plt
from scipy.stats import mstats
import statsmodels.api as sm
import seaborn as sns
from statsmodels.graphics.gofplots import qqline
import math
import warnings
warnings.filterwarnings('ignore')

In [3]:
############################################################################################
# Settings for the calculations that can be changed
lags = 16  # months of lagged FC
Minimum_Number_Stocks = 0.2  # Firm characteritics that are avvailable for at least 20% of stocks

Upper_limit = 0.98       # FC upper outliers
Lower_limit = 1-Upper_limit # FC lower outliers

def winsorize_df(df):
    return df.apply(winsorize_series, axis=0)

def winsorize_series(s):
    return mstats.winsorize(s, limits=[1-Upper_limit, Lower_limit])

In [6]:
############################################################################################
# Read Data

# # Colab 사용할 경우 구글드라이버 폴더에 접근하게 함
from google.colab import drive
drive.mount('/gdrive')
df = pd.read_excel("/gdrive//MyDrive/QuantFinance/QuantDataR.xlsx", None, index_col=0 )

# # 자산의 컴퓨터에 저장된 데이터 읽기
# data= 'QuantDataR.xlsx'
# df = pd.read_excel(data, None, index_col=0 )  # 자신의 컴퓨터에 설치된 파이썬을 사용하는 경우. 파이썬 코드와 데이터 화일의 위치가 동일해야함. 아니면 폴더 경로를 명시

print("==========================================================================")
print("Spreadsheet 확인")
df_names = list(df.keys())
print(df_names)


Mounted at /gdrive
Spreadsheet 확인
['D_Market', 'D_RI', 'D_TradingValue', 'D_P', 'M_Market', 'Beta', 'Volatility', 'DY', 'RI', 'TradingValue', 'Cash', 'AccReceivables', 'Inventory', 'CurrentAsset', 'R&D', 'Plants', 'Intangible', 'Total Asset', 'AccPayable', 'CurrentLiability', 'LongTermDebt', 'TotalLiability', 'CommonEquity', 'TotalShareEqu', 'Sales', 'CoGS', 'DepArm', 'OperatingIncome', 'EBIT', 'Interest', 'NetIncome', 'DividendPerShare', 'EPS', 'CommonShares', 'Investment', 'Employee', 'WorkingCap', 'MarketValue']


In [7]:
# 개별주식의 총수익률 (RI) 데이터 읽음
RID = df['RI']
print("--------------------------------------------------------------------------")
print("데이터형식 확인")
print(RID)

# Dates
date = RID.index
print("--------------------------------------------------------------------------")
print("Dates")
print(date)

#firm names using 'D_Price'
D_Price = df['D_P']
firm_names =  D_Price.columns
print("--------------------------------------------------------------------------")
print("FIrm Names")
print(firm_names)

--------------------------------------------------------------------------
데이터형식 확인
            A SELF-ADMINISTERED REIT.TST. - TOT RETURN IND  \
Name                                                         
2006-08-31                                             NaN   
2006-09-29                                             NaN   
2006-10-31                                             NaN   
2006-11-30                                             NaN   
2006-12-29                                             NaN   
...                                                    ...   
2020-05-29                                          171.99   
2020-06-30                                          167.76   
2020-07-31                                          170.41   
2020-08-31                                          173.85   
2020-09-30                                          198.19   

            ABLE C&C - TOT RETURN IND  AEKYUNG IND - TOT RETURN IND  \
Name                                  

In [8]:
############################################################################################
# Calculation of log-returns

R = []
print("Null stocks: ", end ='')
for i in range(0,RID.shape[1]):

    if RID.shape[0] - RID.iloc[:, i ].isnull().sum() < 2:

        print (i, ", ", end='')
        R_temp = pd.Series([np.nan]*(RID.shape[0]-1))
        R.append(R_temp)
        continue

    lag0 = (np.log(RID.iloc[1:, i ])).reset_index(drop =True)
    lag1 = (np.log(RID.iloc[0:-1, i ])).reset_index(drop =True)


    R_temp =  (lag0-lag1)*100
    R.append(R_temp)


R = pd.concat(R,axis= 1)
R.index = date[1:]
R.columns =firm_names

R = R.replace([np.inf, -np.inf], np.nan)

Null stocks: 25 , 747 , 

In [9]:
############################################################################################
# Firm Characteristics : Among 37 sheets we need 6th~37th sheet

BE = df["CommonEquity"].iloc[1:,:]
BE.columns = firm_names
FC_date = date

BE_L = BE.shift(lags)   # 16달 lag

ME = (df["MarketValue"] *1000).iloc[1:,:]
ME.columns = firm_names
ME_L = ME.shift(1)       # 1달 lag

FC = (BE_L/ME)

FC = FC.replace([np.inf, -np.inf], np.nan)

In [10]:
# Winsorizing FC (to control outliers)
FC = winsorize_df(FC)

# Nomalization
NFC = (FC -np.mean(FC))/np.std(FC)

# # when you want to pass the processing of winszerizing and nomaization
# NFC = FC

# Replacing NA  to 0
NFC = NFC.fillna(0)

# Check the Winsorized and Normalized FC
NFC.describe()


,A SELF-ADMINISTERED REIT.TST.,ABLE C&C,AEKYUNG IND,AIR BUSAN,AJ NETWORKS,AJU CAPITAL,AK HOLDINGS,AK PETROCHEMICAL,ALUKO,AMOREPACIFIC,...,YOUNGONE,YOUNGONE HOLDINGS,YOUNGWIRE,YUANTA SECURITIES KOREA,YUHAN,YUHWA SECURITIES,YUNGJIN PHARM,YUYANG D&U SUSP - SUSP.20/03/20,YUYU PHARMA,ZINUS
count,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,169.000000,...,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02,1.690000e+02
mean,6.306592e-17,4.729944e-17,-2.627747e-17,-6.043818e-17,1.274457e-16,-8.408790e-17,-1.051099e-16,6.832142e-17,2.102197e-17,0.000000,...,-2.364972e-16,1.261318e-16,1.051099e-16,1.997088e-16,7.042361e-16,-2.102197e-16,-1.261318e-16,-1.261318e-16,-4.204395e-17,4.073008e-17
std,7.753647e-01,9.543135e-01,4.295623e-01,3.618734e-01,6.074929e-01,8.997354e-01,9.543135e-01,7.278474e-01,9.543135e-01,0.954314,...,8.625819e-01,9.543135e-01,8.625819e-01,9.543135e-01,9.543135e-01,9.543135e-01,9.543135e-01,9.543135e-01,9.543135e-01,2.672612e-01
min,-2.063870e+00,-1.150785e+00,-1.120421e+00,-8.617563e-01,-1.392170e+00,-1.614095e+00,-1.116963e+00,-1.148354e+00,-1.392595e+00,-1.442129,...,-1.241521e+00,-9.446284e-01,-1.709429e+00,-1.699553e+00,-1.613421e+00,-2.782301e+00,-1.439768e+00,-1.330650e+00,-1.600091e+00,-9.317341e-01
25%,0.000000e+00,-5.929608e-01,0.000000e+00,0.000000e+00,0.000000e+00,-4.737519e-01,-7.205425e-01,-4.046402e-01,-5.846403e-01,-0.601210,...,-7.797348e-01,-6.896527e-01,-4.234302e-01,-6.858088e-01,-6.034202e-01,-5.494036e-01,-1.005361e+00,-8.867036e-01,-8.122207e-01,0.000000e+00
50%,0.000000e+00,-2.988863e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.973516e-01,0.000000e+00,0.000000e+00,-0.173015,...,0.000000e+00,-2.043928e-01,0.000000e+00,-1.210378e-01,0.000000e+00,-5.423028e-03,6.754274e-02,-4.506304e-02,0.000000e+00,0.000000e+00
75%,3.336747e-01,2.875324e-01,0.000000e+00,0.000000e+00,0.000000e+00,6.497242e-01,3.206017e-01,0.000000e+00,3.670567e-01,0.400838,...,4.220836e-01,3.045890e-01,4.934696e-01,6.848882e-01,4.012487e-01,6.715183e-01,6.540621e-01,5.142527e-01,7.731522e-01,0.000000e+00
max,2.260398e+00,4.003719e+00,1.812802e+00,3.493795e+00,2.624320e+00,2.397226e+00,3.055377e+00,2.751087e+00,4.438414e+00,2.981767,...,3.251231e+00,3.813128e+00,3.174423e+00,1.998367e+00,3.099166e+00,1.992091e+00,2.124597e+00,3.446040e+00,2.202487e+00,1.296875e+00


In [11]:
len(NFC.T)

792

In [12]:
results = pd.DataFrame(np.empty((NFC.shape[0],3))*np.nan, index=R.index, columns=['Alpha','Beta','R2'])
Alpha = []
Beta = []
R2 = []


for t in range(lags+1,NFC.shape[0]):

    if NFC.shape[1]-(NFC.iloc[t-1,:]==0).sum()>100:
        XX = sm.add_constant(NFC.iloc[t-1,:]) #### warrning

        #There are object columns in df ,so convert these to float
        XX = pd.DataFrame(XX, dtype='float')

        Y = R.iloc[t,:] # returns of individual stocks at time t

        # remove omit value
        sdata = pd.concat([XX,Y],axis = 1 )
        sdata = sdata.dropna(axis=0).reset_index(drop =True)
        sdata.columns = ['Const','BM','RET']

        XX = sdata[['Const','BM']]

        Y = sdata.loc[:,'RET']

        coef = np.linalg.inv(XX.T @ XX) @ XX.T @ Y

        Alpha.append(coef[0])
        Beta.append(coef[1])

        resid  = Y - XX @ coef.values
        r2 =  1- (resid.T @ resid)  / (sdata.shape[0]-2) / np.var(Y)
        R2.append(r2)

        results.iloc[t,:] = [coef[0], coef[1], r2]

# results = pd.concat([pd.Series(Alpha),
                #    pd.Series(Beta),
                #    pd.Series(R2)], axis=1)

results

,Alpha,Beta,R2
Name,,,
2006-09-29,NaN,NaN,NaN
2006-10-31,NaN,NaN,NaN
2006-11-30,NaN,NaN,NaN
2006-12-29,NaN,NaN,NaN
2007-01-31,NaN,NaN,NaN
...,...,...,...
2020-05-29,5.374014,-0.558111,0.000951
2020-06-30,-1.187236,-0.378155,-0.001221
2020-07-31,7.247540,-0.609354,0.001600


In [13]:
print("Alpha")
print("Mean:",results["Alpha"].dropna().mean(), "STD_M:",results["Alpha"].dropna().std() /math.sqrt(results["Alpha"].dropna().shape[0]))
print("Beta")
print("Mean:",results["Beta"].dropna().mean(), "STD_M:",results["Beta"].dropna().std() /math.sqrt(results["Beta"].dropna().shape[0]))
print("R2: ", results["R2"].dropna().mean())


Alpha
Mean: 0.14764954434636368 STD_M: 0.48623276738038634
Beta
Mean: 1.1713988420196408 STD_M: 0.11176560676520068
R2:  0.014693398656512272


In [14]:
########################################################################
# Identification of Factors
#######################################################################

RHPS = pd.DataFrame(np.empty((NFC.shape[0],11))*np.nan, index=R.index, columns=['Number_Stocks','High_BM','2','3','Low_BM','RET_High_BM','2','3','4','RET_Low_BM','HML'])

quantile_FC =  NFC.quantile(q=[.2, .4, .6 , .8 ], interpolation='nearest', axis=1).T

for t in range(lags+1, NFC.shape[0]-1):

        if (NFC.shape[1]- NFC.iloc[t,:].isnull().sum())>100:

            RHPS.iloc[t,0] = (NFC.shape[1]- NFC.iloc[t,:].isnull().sum())  # the number of stocks that are used to form portfolios
            Y = R.iloc[t+1,:]
            RHPS.iloc[t,1:5] = quantile_FC.iloc[t,:]

            L = NFC.iloc[t,:] <= quantile_FC.iloc[t,0]
            H = NFC.iloc[t,:] >= quantile_FC.iloc[t,3]
            RHPS.iloc[t+1,10] = Y[H].dropna().mean()  - Y[L].dropna().mean()

            RHPS.iloc[t,1] = quantile_FC.iloc[t,3]
            RHPS.iloc[t,2] = quantile_FC.iloc[t,2]
            RHPS.iloc[t,3] = quantile_FC.iloc[t,1]
            RHPS.iloc[t,4] = quantile_FC.iloc[t,0]

            P1 = (NFC.iloc[t,:] >= quantile_FC.iloc[t,3])
            P2 = (NFC.iloc[t,:] > quantile_FC.iloc[t,2]) & (NFC.iloc[t,:] <= quantile_FC.iloc[t,3])
            P3 = (NFC.iloc[t,:] > quantile_FC.iloc[t,1]) & (NFC.iloc[t,:] <= quantile_FC.iloc[t,2])
            P4 = (NFC.iloc[t,:] > quantile_FC.iloc[t,0]) & (NFC.iloc[t,:] <= quantile_FC.iloc[t,1])
            P5 = (NFC.iloc[t,:] <= quantile_FC.iloc[t,0])


            RHPS.iloc[t+1,5] =  Y[P1].dropna().mean()
            RHPS.iloc[t+1,6] =  Y[P2].dropna().mean()
            RHPS.iloc[t+1,7] =  Y[P3].dropna().mean()
            RHPS.iloc[t+1,8] =  Y[P4].dropna().mean()
            RHPS.iloc[t+1,9] =  Y[P5].dropna().mean()

RHPS

,Number_Stocks,High_BM,2,3,Low_BM,RET_High_BM,2,3,4,RET_Low_BM,HML
Name,,,,,,,,,,,
2006-09-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-11-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-05-29,792.0,2.021670,1.243342,0.236082,-0.489340,3.859066,3.991987,4.767876,7.409009,4.561384,-0.702318
2020-06-30,792.0,2.260325,1.242055,0.264655,-0.501261,-3.573758,0.317528,-1.604039,-0.701734,-1.806522,-1.767236
2020-07-31,792.0,1.996687,1.051609,0.062803,-0.636533,6.768432,6.013986,5.288213,6.425574,9.134824,-2.366392


In [17]:
Mean = RHPS["2008-03":"2020-07"].mean(axis=0 )
STD_M = (RHPS["2008-03":"2020-07"].std() /math.sqrt(RHPS["2008-03":"2020-07"].shape[0]-lags-1))
t_statistic = Mean/STD_M

result = pd.concat([Mean,STD_M,t_statistic], axis=1)
result.columns = ['Mean','STD_M','t_statistic']
result.T

,Number_Stocks,High_BM,2,3,Low_BM,RET_High_BM,2,3,4,RET_Low_BM,HML
Mean,792.0,0.644480,0.121777,-0.229805,-0.723918,1.322764,0.476029,0.091569,-0.063982,-1.432192,2.754955
STD_M,0.0,0.047305,0.030229,0.021983,0.018435,0.553603,0.530915,0.593001,0.556883,0.560331,0.283895
t_statistic,inf,13.624061,4.028531,-10.453542,-39.268349,2.389371,0.896619,0.154416,-0.114893,-2.555974,9.704137


In [19]:
# Read Data for the Market Return and Risk-free rate

Mdata = pd.read_csv("/gdrive//MyDrive/QuantFinance/data.csv",index_col=0 )
Mdata.index = pd.to_datetime(Mdata.index)
Mdata = Mdata.dropna(axis= 0)
n = Mdata.shape[0]
MR = Mdata.loc["2008-03":"2020-07","RKOSPI"]
RF = Mdata.loc["2008-03":"2020-07","RF_M"]


Y = RHPS.loc["2008-03":"2020-07",'HML']
X=  pd.concat([MR-RF],axis = 1)
model = sm.OLS(Y,sm.add_constant(X))
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    HML   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.1236
Date:                Sat, 23 Sep 2023   Prob (F-statistic):              0.726
Time:                        05:46:06   Log-Likelihood:                -387.01
No. Observations:                 149   AIC:                             778.0
Df Residuals:                     147   BIC:                             784.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.7550      0.268     10.280      0.0